This notebook downsamples reanalysis dataset from 8,11 to 96,132

In [1]:
import xarray as xr
# import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
# import cartopy.crs as ccrs  # for plotting map
# import cartopy
# import matplotlib as mpl
import numpy as np
import scipy.ndimage

# import tensorflow as tf
from functions import *

c:\Users\bobby\anaconda3\envs\research\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# raeding reanalysis data
gefsdir = 'C:\\Users\\bobby\\Documents\\GitHub\\climateMedium\\raw_data\\'
reanalysis = xr.open_dataset(gefsdir+'GEFSv12-Reanalysis_tp_2000_2019.nc').sel(lon=slice('102.5', '105.00'), lat=('2.5','2.25','2','1.75','1.5','1.25','1','0.75'))
reanalysis = reanalysis.isel(time=slice(1, None)) 
del reanalysis.attrs['history'] # remove long history text
lon = reanalysis.lon.values
lat = reanalysis.lat.values
time = np.arange(np.datetime64("2000-01-01T06"), np.datetime64("2020-01-01T00"), np.timedelta64(6, "h"))

In [3]:
reanalysis

<xarray.Dataset>
Dimensions:  (time: 29219, lon: 11, lat: 8)
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01T06:00:00 ... 2019-12-31T18:00:00
  * lon      (lon) float64 102.5 102.8 103.0 103.2 ... 104.2 104.5 104.8 105.0
  * lat      (lat) float64 2.5 2.25 2.0 1.75 1.5 1.25 1.0 0.75
Data variables:
    tp       (time, lat, lon) float32 ...
Attributes:
    CDI:          Climate Data Interface version 1.9.9rc1 (https://mpimet.mpg...
    Conventions:  CF-1.6
    CDO:          Climate Data Operators version 1.9.9rc1 (https://mpimet.mpg...

In [31]:
# IF REQUIRED TO TAKE SAMPLE

# wrf = wrf.isel(Time=slice(0,6000))
# reanalysis = reanalysis.isel(time=slice(0,6000))
# reforecast_apcp_c00 = reforecast_apcp_c00.isel(time=slice(0,6000))

In [4]:
# bilinear interpolation for lat/lon
bilinear_lon = scipy.ndimage.zoom(lon, 12, order=1)
bilinear_lat = scipy.ndimage.zoom(lat, 12, order=1)
row_meshgrid, col_meshgrid = np.meshgrid(bilinear_lat, bilinear_lon, indexing='ij')

# bilinear interpolation for the reanalysis values
x = reanalysis.tp.values
x.shape
bilinear_gefs = scipy.ndimage.zoom(x, (1,12,12), order=1)

downscaled_reanalysis = xr.Dataset(
                data_vars=dict(
                    tp=(["time", "lon","lat"], bilinear_gefs, {"units":"kgm**-2"})
                ),
                coords=dict(
                    time=(["time"], time),
                    xlon=(["lon","lat"], col_meshgrid),
                    xlat=(["lon","lat"], row_meshgrid),
                ),
                attrs=dict(description="coords with matrices"),
            )

downscaled_reanalysis = downscaled_reanalysis.swap_dims({"lon":"lat", "lat":"lon"})

In [5]:
y_train_hr = downscaled_reanalysis.sel(time=slice('2000-01-01T06', '2014-01-01T06'))
y_val_hr = downscaled_reanalysis.sel(time=slice('2014-01-01T12', '2016-12-31T12'))
y_test_hr = downscaled_reanalysis.sel(time=slice('2016-12-31T18', '2019-12-31T18'))

In [9]:
y_train_hr

<xarray.Dataset>
Dimensions:  (time: 20457, lat: 96, lon: 132)
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01T06:00:00 ... 2014-01-01T06:00:00
    xlon     (lat, lon) float64 102.5 102.5 102.5 102.6 ... 105.0 105.0 105.0
    xlat     (lat, lon) float64 2.5 2.5 2.5 2.5 2.5 ... 0.75 0.75 0.75 0.75 0.75
Dimensions without coordinates: lat, lon
Data variables:
    tp       (time, lat, lon) float32 1.9 2.114 2.327 2.541 ... 1.204 0.9519 0.7
Attributes:
    description:  coords with matrices

In [34]:
def timecheck(train, val, test):
    '''
    Return missing time step in train, val and test split
    '''
    train_timecheck = np.arange(np.datetime64("2000-01-01T06"), np.datetime64("2014-01-01T12"), np.timedelta64(6, "h"))
    val_timecheck = np.arange(np.datetime64("2014-01-01T12"), np.datetime64("2016-12-31T18"), np.timedelta64(6, "h"))
    test_timecheck = np.arange(np.datetime64("2016-12-31T18"), np.datetime64("2020-01-01T00"), np.timedelta64(6, "h"))
    print('train:', set(train_timecheck) - set(train.time.values.astype('datetime64[h]')))
    print('val:', set(val_timecheck) - set(val.time.values.astype('datetime64[h]')))
    print('test:', set(test_timecheck) - set(test.time.values.astype('datetime64[h]')))
    return

timecheck(y_train_hr, y_val_hr, y_test_hr) # no missing dates

train: set()
val: set()
test: set()


In [35]:
y_train_hr

<xarray.Dataset>
Dimensions:  (time: 20457, lat: 96, lon: 132)
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01T06:00:00 ... 2014-01-01T06:00:00
    xlon     (lat, lon) float64 102.5 102.5 102.5 102.6 ... 105.0 105.0 105.0
    xlat     (lat, lon) float64 2.5 2.5 2.5 2.5 2.5 ... 0.75 0.75 0.75 0.75 0.75
Dimensions without coordinates: lat, lon
Data variables:
    tp       (time, lat, lon) float32 1.9 2.114 2.327 2.541 ... 1.204 0.9519 0.7
Attributes:
    description:  coords with matrices

In [36]:
scaler_train_tp = MinMaxScaler() 
y_one_col = y_train_hr.tp.values.reshape([y_train_hr.tp.values.shape[0]*y_train_hr.tp.values.shape[1]*y_train_hr.tp.values.shape[2], 1])
y_one_col = np.log10(y_one_col+1) # 10**X_one_col - 1 to scale back
y_one_col_res = scaler_train_tp.fit_transform(y_one_col) # scaler_train_apcp.inverse_transform(X_one_col_res) to scale back, or use 10**scaler_train_apcp.inverse_transform(X_one_col_res) -1 only
y_train_hr.tp.values = y_one_col_res.reshape(y_train_hr.tp.values.shape)

In [37]:
def transform_val_test(val_test, scaler_train, is_prec=True):
    '''
    Input (example): ds_val_apcp.tp, scaler_train_apcp, True/False
    Output: Transformed validation/test XR data
    If is_prec set to True, variable is precipitation
    '''
    if is_prec == True:
        X_one_col = val_test.values.reshape([val_test.values.shape[0]*val_test.values.shape[1]*val_test.values.shape[2], 1])
        X_one_col = np.log10(X_one_col+1) 
        X_one_col_res = scaler_train.transform(X_one_col) 
        val_test.values = X_one_col_res.reshape(val_test.values.shape)
        return val_test.values
        
    else:
        X_one_col = val_test.values.reshape([val_test.values.shape[0]*val_test.values.shape[1]*val_test.values.shape[2], 1])
        # X_one_col = np.log10(X_one_col+1) 
        X_one_col_res = scaler_train.transform(X_one_col) 
        val_test.values = X_one_col_res.reshape(val_test.values.shape)
        return val_test.values

In [38]:
# reanalysis
y_val_hr.tp.values = transform_val_test(y_val_hr.tp, scaler_train_tp, True)
y_test_hr.tp.values = transform_val_test(y_test_hr.tp, scaler_train_tp, True)

In [39]:
y_val_hr

<xarray.Dataset>
Dimensions:  (time: 4381, lat: 96, lon: 132)
Coordinates:
  * time     (time) datetime64[ns] 2014-01-01T12:00:00 ... 2016-12-31T12:00:00
    xlon     (lat, lon) float64 102.5 102.5 102.5 102.6 ... 105.0 105.0 105.0
    xlat     (lat, lon) float64 2.5 2.5 2.5 2.5 2.5 ... 0.75 0.75 0.75 0.75 0.75
Dimensions without coordinates: lat, lon
Data variables:
    tp       (time, lat, lon) float32 0.2273 0.2262 0.2251 ... 0.1303 0.123
Attributes:
    description:  coords with matrices

In [41]:
# training FOR REANALYSIS DOWNSCALINGs
y_hr_train = y_train_hr.tp.values
y_hr_train = y_hr_train[..., np.newaxis]
print(y_hr_train.shape)
np.save('C:\\Users\\bobby\\Documents\\GitHub\\climateMedium\\postprocessed_data\\y_hr_train.npy', y_hr_train)

# val
y_hr_val = y_val_hr.tp.values
y_hr_val = y_hr_val[..., np.newaxis]
print(y_hr_val.shape)
np.save('C:\\Users\\bobby\\Documents\\GitHub\\climateMedium\\postprocessed_data\\y_hr_val.npy', y_hr_val)

# testing
y_hr_test = y_test_hr.tp.values
y_hr_test = y_hr_test[..., np.newaxis]
print(y_hr_test.shape)
np.save('C:\\Users\\bobby\\Documents\\GitHub\\climateMedium\\postprocessed_data\\y_hr_test.npy', y_hr_test)

(20457, 96, 132, 1)
(4381, 96, 132, 1)
(4381, 96, 132, 1)
